In [2]:
import numpy as np
import sys
#np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4
from helper_PFCI import PFHamiltonianGenerator
from helper_PFCI import Determinant
from helper_cqed_rhf import cqed_rhf
np.set_printoptions(threshold=sys.maxsize)

In [7]:
weights = np.array([1, 3, 1, 6, 1, 1])
norm = np.sum(weights)
norm_weights = weights / norm
print(norm)
print(np.sum(norm_weights))

13
1.0


In [ ]:
mol_str = """
0 1
O
H 1 1.0
H 1 1.0 2 104.0
symmetry c1
"""

options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}

cavity_dict = {
    'omega_value' : 0.0,
    'lambda_vector' : np.array([0, 0, 0]),
    'ci_level' : 'cis',
    'ignore_coupling' : False,
    'number_of_photons' : 1,
    'natural_orbitals' : False,
    'nact_orbs' : 0,
    'nact_els' : 0
}

mol = psi4.geometry(mol_str)

psi4.set_options(options_dict)
psi4.core.set_output_file("output.dat", False)


test_pf = PFHamiltonianGenerator(
    mol_str,
    options_dict,
    cavity_dict
)

e_fci, wavefunctions = np.linalg.eigh(test_pf.H_PF)

test_pf.calc1RDMfromCIS(wavefunctions[:, 14])

# test traces of different blocks
expected_trace_Dij = 9.0
expected_trace_Dab = 1.0
expected_trace_D1 = 10.0
expected_trace_D1_spatial = 10.0

trace_Dij = np.trace(test_pf.Dij)
trace_Dab = np.trace(test_pf.Dab)
trace_D1 = np.trace(test_pf.D1)
trace_D1_spatial = np.trace(test_pf.D1_spatial)

assert np.isclose(expected_trace_D1, trace_D1)
assert np.isclose(expected_trace_D1_spatial, trace_D1_spatial)
assert np.isclose(expected_trace_Dab, trace_Dab)
assert np.isclose(expected_trace_Dij, trace_Dij)

# test trace of K1 D1 against <C|1H|C>
e1_test_rdm = np.einsum("pq,pq->", test_pf.Hspin, test_pf.D1)

# 1H @ C
temp = np.einsum("pq,q->p", test_pf.H_1E, wavefunctions[:, 14])
e1_test_wfn = np.dot(wavefunctions[:, 14].T, temp)

expected_1e_energy = e1_test_wfn - test_pf.Enuc

assert np.isclose(e1_test_rdm, expected_1e_energy)



In [ ]:
eig, vec = np.linalg.eigh(test_pf.D1_spatial)
idx = eig.argsort()[::-1]   
eigenValues = eig[idx]
eigenVectors = vec[:,idx]

In [ ]:
print(eigenValues)

In [ ]:
print(eigenVectors)